In [ ]:
# npx hardhat node --fork https://eth-mainnet.alchemyapi.io/v2/xOu9KmQYmgmqBuYhhPW0naOB9YRY3foa
# ngrok authtoken 1WA2UnsI5GHgexqADDhvw7v1Npd_dqSFiM3bX98ny1vjvx3a
# ngrok http 8545

# 初始化
require 'ethereum.rb'
require 'eth'
require 'faraday'
require 'parallel'
require 'resolv-replace'
require 'erb'

false

In [ ]:
#configure
etherscan_apikey = "545QYEP8TJKJWMZ7QP41QYXZNMJDVCJUI9"

# 参数设置
fomo_name = "FOMOFast"
fomo_addr = "0x324b50cAA2b95df219e78e016A3014E9B70EF5a2".downcase

playerbook_name = "PlayerBook"
playerbook_addr = "0x18A586F55DA3C95Fe1bD79A4E54F9a37987b688A".downcase


# 定义账户
sender_account = "0xaCC961474eb17C103BBed96FC8f0E4eEac91c475"
sender_priv_key = "6ac277496bb17f43074545db8e2e26e903aa014687448d8ed564d9881ce9bda9"
sender_key = Eth::Key.new priv: sender_priv_key

impersonate_account = "0xf39fd6e51aad88f6f4ce6ab8827279cfffb92266"
impersonate_priv_key = "ac0974bec39a17e36ba4a6b4d238ff944bacb478cbed5efcae784d7bf4f2ff80"
impersonate_key = Eth::Key.new priv: impersonate_priv_key


# 创建Client
$client = Ethereum::HttpClient.new("http://4ce1-211-154-194-206.ngrok.io")
$client.gas_limit = 2_000_000
$client.gas_price = $client.eth_gas_price["result"].to_i(16)

# 创建Fomo Contract
fomo_abi_url = "https://api.etherscan.io/api?module=contract&action=getabi&address=#{fomo_addr}&apikey=#{etherscan_apikey}"
fomo_abi = JSON.parse(Faraday.get(fomo_abi_url).body)["result"]

$fomo_contract = Ethereum::Contract.create(
        client: $client, 
        name: fomo_name, 
        address: fomo_addr, 
        abi: fomo_abi)

$fomo_contract.key = sender_key
$fomo_contract.gas_limit = 2_000_000
$fomo_contract.gas_price = $client.eth_gas_price["result"].to_i(16)


# 创建PlayerBook Contract
playerbook_abi_url = "https://api.etherscan.io/api?module=contract&action=getabi&address=#{playerbook_addr}&apikey=#{etherscan_apikey}"
playerbook_abi = JSON.parse(Faraday.get(playerbook_abi_url).body)["result"]

$playerbook_contract = Ethereum::Contract.create(
        client: $client, 
        name: playerbook_name, 
        address: playerbook_addr, 
        abi: playerbook_abi)

$playerbook_contract.key = sender_key
$playerbook_contract.gas_limit = 2_000_000
$playerbook_contract.gas_price = $client.eth_gas_price["result"].to_i(16)

nil

In [ ]:
def get_balance(addr)
    $client.eth_get_balance(addr)["result"].to_i(16)/1e18.to_f
end

:get_balance

In [ ]:
# ===== View Account Balance =======

accounts = [sender_account,impersonate_account]

output = accounts.map do |addr|
  [addr, get_balance(addr)].join(" - ")
end.join("\n")

puts output

0xaCC961474eb17C103BBed96FC8f0E4eEac91c475 - 100.0
0xf39fd6e51aad88f6f4ce6ab8827279cfffb92266 - 9900.00028990281


In [ ]:
## 操作账户转账到Sender account = 100 ETH
sender_balance = get_balance(sender_account)
impersonate_balance =  get_balance(impersonate_account)
sender_balance = amount = 100.to_i-sender_balance
$client.transfer(impersonate_key, sender_account, amount) if amount>0


In [ ]:
# === Inspect PlayerBook ===


puts "==Read=="
puts $playerbook_contract.abi.filter{ |x| x["constant"]==true }
    .filter{ |x| x["type"]=="function" }
    .map {|x| [x["type"],x["name"],x["inputs"].to_s,x["outputs"].to_s].join(" - ") }.join("\n")

puts "==Write=="
puts $playerbook_contract.abi.filter{ |x| x["constant"]==false }
    .filter{ |x| x["type"]=="function" }
    .map {|x| [x["type"],x["name"],x["inputs"].to_s,x["outputs"].to_s].join(" - ") }.join("\n")

==Read==
function - pIDxAddr_ - [{"name"=>"", "type"=>"address"}] - [{"name"=>"", "type"=>"uint256"}]
function - registrationFee_ - [] - [{"name"=>"", "type"=>"uint256"}]
function - getNameFee - [] - [{"name"=>"", "type"=>"uint256"}]
function - plyrNames_ - [{"name"=>"", "type"=>"uint256"}, {"name"=>"", "type"=>"bytes32"}] - [{"name"=>"", "type"=>"bool"}]
function - gameNames_ - [{"name"=>"", "type"=>"address"}] - [{"name"=>"", "type"=>"bytes32"}]
function - pIDxName_ - [{"name"=>"", "type"=>"bytes32"}] - [{"name"=>"", "type"=>"uint256"}]
function - pID_ - [] - [{"name"=>"", "type"=>"uint256"}]
function - getPlayerAddr - [{"name"=>"_pID", "type"=>"uint256"}] - [{"name"=>"", "type"=>"address"}]
function - checkIfNameValid - [{"name"=>"_nameStr", "type"=>"string"}] - [{"name"=>"", "type"=>"bool"}]
function - getPlayerName - [{"name"=>"_pID", "type"=>"uint256"}] - [{"name"=>"", "type"=>"bytes32"}]
function - plyrNameList_ - [{"name"=>"", "type"=>"uint256"}, {"name"=>"", "type"=>"uint256"}

In [ ]:
gID_ = $playerbook_contract.call.g_id_
pID_ = $playerbook_contract.call.p_id_
registrationFee_ = $playerbook_contract.call.registration_fee_
getNameFee = $playerbook_contract.call.get_name_fee

addrs = (1..pID_).map do |i|
    {pid:i,addr:$playerbook_contract.call.get_player_addr(i)}
end

games = (1..gID_).map do |i|
    {gid:i, addr:$playerbook_contract.call.games_(i)}
end

[{:gid=>1, :addr=>"324b50caa2b95df219e78e016a3014e9b70ef5a2"}, {:gid=>2, :addr=>"78d4883aece4a783a201a39c69d79dd605b18895"}]

In [ ]:
# 查询合约情况

puts "gID_ = #{gID_}"
puts "pID_ = #{pID_}"
puts "registrationFee_ = #{registrationFee_}"
puts "getNameFee = #{getNameFee}"



puts
puts "player addr:"
puts addrs.map {|x| 
    [x[:addr],
     $playerbook_contract.call.get_player_name(x[:pid]),
     $playerbook_contract.call.get_player_l_aff(x[:pid]) 
     ]
     .join(" - ") }.join("\n")
puts "\n"
puts "game addr:"
puts games.map {|x| 
    [x[:addr],
     $playerbook_contract.call.game_names_(x[:addr]),
     get_balance("0x"+x[:addr])]
    .join(" - ")}.join("\n")


gID_ = 2
pID_ = 9
registrationFee_ = 15792089237316195423570985008687907853269984665640564039457584007913129639936
getNameFee = 15792089237316195423570985008687907853269984665640564039457584007913129639936

player addr:
acc961474eb17c103bbed96fc8f0e4eeac91c475 - masto - 0
b74c54ebc1b94dfe93711780f9087f2ba360b70f - admino - 0
718b6aca52548416e27ab38699cbc4c0ed304b95 - tothemoon - 0
838939a62daf9594fe955a9a2ccd735a35e153a1 -  - 0
50d625fcffc6ffb42f10e6b8b87f107f5d9a2e30 - kevingy - 0
a41d1a5d21cc616b52ae2e3e2224768eba3f81ff -  - 0
7f2f933ee22b802935449911fc8c7f35fb00409c -  - 0
663c9a828a7b5d7821e709714c29357506102e49 -  - 0
e50ac0d497db44ffaaeb7d98cb57c420992e1d9d -  - 0

game addr:
324b50caa2b95df219e78e016a3014e9b70ef5a2 - fomofast - 0.05611162315356991
78d4883aece4a783a201a39c69d79dd605b18895 - agent232attack - 0.0


In [ ]:
# === Inspect Fomo ===


puts "==Read=="
puts $fomo_contract.abi.filter{ |x| x["constant"]==true }
    .filter{ |x| x["type"]=="function" }
    .map {|x| [x["type"],x["name"],x["inputs"].to_s,x["outputs"].to_s].join(" - ") }.join("\n")

puts "==Write=="
puts $fomo_contract.abi.filter{ |x| x["constant"]==false }
    .filter{ |x| x["type"]=="function" }
    .map {|x| [x["type"],x["name"],x["inputs"].to_s,x["outputs"].to_s].join(" - ") }.join("\n")

==Read==
function - getBuyPrice - [] - [{"name"=>"", "type"=>"uint256"}]
function - name - [] - [{"name"=>"", "type"=>"string"}]
function - pIDxAddr_ - [{"name"=>"", "type"=>"address"}] - [{"name"=>"", "type"=>"uint256"}]
function - airDropTracker_ - [] - [{"name"=>"", "type"=>"uint256"}]
function - round_ - [{"name"=>"", "type"=>"uint256"}] - [{"name"=>"plyr", "type"=>"uint256"}, {"name"=>"team", "type"=>"uint256"}, {"name"=>"end", "type"=>"uint256"}, {"name"=>"ended", "type"=>"bool"}, {"name"=>"strt", "type"=>"uint256"}, {"name"=>"keys", "type"=>"uint256"}, {"name"=>"eth", "type"=>"uint256"}, {"name"=>"pot", "type"=>"uint256"}, {"name"=>"mask", "type"=>"uint256"}, {"name"=>"ico", "type"=>"uint256"}, {"name"=>"icoGen", "type"=>"uint256"}, {"name"=>"icoAvg", "type"=>"uint256"}]
function - plyrNames_ - [{"name"=>"", "type"=>"uint256"}, {"name"=>"", "type"=>"bytes32"}] - [{"name"=>"", "type"=>"bool"}]
function - fees_ - [{"name"=>"", "type"=>"uint256"}] - [{"name"=>"gen", "type"=>"uint25

In [ ]:
$fomo_contract.abi.filter{ |x| x["constant"]==true }
    .filter{ |x| x["type"]=="function" }
    .filter{ |x| x["inputs"]==[]}.map {|x| x["name"]}.map { |x| 
        puts "#{x} = #{$fomo_contract.call.send(x.underscore)}"
    }

puts

getBuyPrice = 75000000000000
name = FOMO Fast
airDropTracker_ = 0
rID_ = 9
getCurrentRoundInfo = [0, 9, 0, 1533126208, 1533122608, 287426290807430, 0, "0000000000000000000000000000000000000000", "", 0, 0, 0, 0, 4096196959362189000]
symbol = Fast
getTimeLeft = 0
activated_ = true
airDropPot_ = 4096196959362189



In [ ]:
puts "player addr:"
table = addrs.map {|x| 
     a=$fomo_contract.call.get_player_info_by_address(x[:addr])
     b=$fomo_contract.call.get_player_vaults(x[:pid])
     c=$fomo_contract.call.plyr_(x[:pid]) 

    [x[:addr],a[1],a[2],a[3],a[4],a[5],a[6],b[0],b[1],b[2],c[5],c[6]
    ].map {|y| "<td>#{y}</td>"}.join }.map {|y| "<tr>#{y}</tr>"}.join("\n")

header = %w(addr name keys  winnings  general  affiliate  round_eth winnings  general  affiliate lrnd laff).map { |x|
    "<td>#{x}</td>"
}.join

table = "<table><tr>#{header}</tr>#{table}</table>"

IRuby.html table


player addr:


addr,name,keys,winnings,general,affiliate,round_eth,winnings,general,affiliate,lrnd,laff
acc961474eb17c103bbed96fc8f0e4eeac91c475,masto,0,230400000000002,549999999999993,0,0,230400000000002,549999999999993,0,1,0
b74c54ebc1b94dfe93711780f9087f2ba360b70f,admino,0,0,0,0,0,0,0,0,0,0
718b6aca52548416e27ab38699cbc4c0ed304b95,tothemoon,0,0,75000000000100,0,0,0,75000000000100,0,5,0
838939a62daf9594fe955a9a2ccd735a35e153a1,,0,0,0,0,0,0,0,0,5,0
50d625fcffc6ffb42f10e6b8b87f107f5d9a2e30,kevingy,0,0,602337554205766,0,0,0,602337554205766,0,2,0
a41d1a5d21cc616b52ae2e3e2224768eba3f81ff,,0,0,0,0,0,0,0,0,5,0
7f2f933ee22b802935449911fc8c7f35fb00409c,,0,0,0,0,0,0,0,0,8,0
663c9a828a7b5d7821e709714c29357506102e49,,0,5475384783502422,6279518822969440,0,0,5475384783502422,6279518822969440,0,7,0
e50ac0d497db44ffaaeb7d98cb57c420992e1d9d,,0,0,22582054687500100,0,0,0,22582054687500100,0,0,0


In [ ]:
rID_ = $fomo_contract.call.r_id_
round = (1..rID_).map do |i|
    $fomo_contract.call.round_(i)
end

nil

In [ ]:
round_team = (1..rID_).map do |i|
    [$fomo_contract.call.rnd_tm_eth_(i,0),$fomo_contract.call.rnd_tm_eth_(i,1),$fomo_contract.call.rnd_tm_eth_(i,2),$fomo_contract.call.rnd_tm_eth_(i,3)]
end

nil

In [ ]:
round_team
# puts round_team.map do |x| x.join(" - ") end.join("\n")

[[0, 1000000000000000, 0, 0], [37725150625000404, 0, 53439894843750201, 0], [7575789062500100, 0, 0, 0], [158118438279967717, 0, 0, 0], [136382219531250504, 0, 0, 0], [0, 0, 75000000000000, 0], [15228203593750100, 0, 0, 0], [0, 0, 75000000000000, 0], [0, 0, 0, 0]]

In [ ]:
html = """
  <table>
    <tr>
<% %w(plyr team team0 team1 team2 team3  end  ended strt duration keys eth keys_per_eth pot mask ico icoGen icoAvg).each do |x| %>
        <th><%= x %></th>
<% end %>
    </tr>
    <% acc_eth = 0
       round.each_with_index do |rnd,i| 
       %>
    <tr>
            <td> <%= rnd[0] %> </td>
            <td> <%= rnd[1] %> </td>
            <td> <%= round_team[i][0]/1e18 %> </td>
            <td> <%= round_team[i][1]/1e18 %> </td>
            <td> <%= round_team[i][2]/1e18 %> </td>
            <td> <%= round_team[i][3]/1e18 %> </td>
            <td> <%= Time.at(rnd[2]).to_fs(:db) %> </td>
            <td> <%= rnd[3] %> </td>
            <td> <%= Time.at(rnd[4]).to_fs(:db) %> </td>
            <td> <%= ActiveSupport::Duration.build(rnd[2]-rnd[4]).inspect %> </td>
            <td> <%= rnd[5]/1e18 %> </td>
            <td> <%= rnd[6]/1e18 %> </td>
            <td> <%= rnd[6]==0 ? 0 : rnd[5]/rnd[6] %> </td>
            <td> <%= rnd[7]/1e18 %> </td>
            <td> <%= rnd[8] %> </td>
            <td> <%= rnd[9] %> </td>
            <td> <%= rnd[10] %> </td>
            <td> <%= rnd[11] %> </td>
    </tr>
    <% 
       end %>
  </table>
"""

IRuby.html ERB.new(html).result(binding)

plyr,team,team0,team1,team2,team3,end,ended,strt,duration,keys,eth,keys_per_eth,pot,mask,ico,icoGen,icoAvg
1,1,0.0,0.001,0.0,0.0,2018-07-23 01:05:14,true,2018-07-22 16:20:30,"8 hours, 44 minutes, and 44 seconds",13.333162041616681,0.001,13333,0.000480000000000005,41250529940556,0,0,0
3,0,0.0377251506250004,0.0,0.0534398948437502,0.0,2018-07-23 08:20:53,true,2018-07-23 05:21:45,"2 hours, 59 minutes, and 8 seconds",1214.0000000000082,0.09116504546875061,13316,0.039934183606409934,58188305037109,0,0,0
3,0,0.0075757890625001,0.0,0.0,0.0,2018-07-23 09:44:29,true,2018-07-23 08:27:39,"1 hour, 16 minutes, and 50 seconds",101.00000000000132,0.0075757890625001,13331,0.014597201440665824,43149021883897,0,0,0
4,0,0.15811843827996772,0.0,0.0,0.0,2018-07-23 14:31:31,true,2018-07-23 12:18:35,"2 hours, 12 minutes, and 56 seconds",2103.6383534373826,0.15811843827996772,13304,0.09994342927267007,51973224498368,0,0,0
4,0,0.1363822195312505,0.0,0.0,0.0,2018-07-23 18:10:42,true,2018-07-23 14:49:47,"3 hours, 20 minutes, and 55 seconds",1815.0000000000068,0.1363822195312505,13308,0.10804262901270416,55507865772549,0,0,0
7,2,0.0,0.0,7.5e-05,0.0,2018-07-26 11:46:10,true,2018-07-24 02:02:39,"2 days, 9 hours, 43 minutes, and 31 seconds",1.0,7.5e-05,13333,0.028321629027138192,5191387095843308,0,0,0
8,0,0.0152282035937501,0.0,0.0,0.0,2018-07-27 15:16:18,true,2018-07-26 12:05:07,"1 day, 3 hours, 11 minutes, and 11 seconds",203.00000000000134,0.0152282035937501,13330,0.011977404213911549,30933590260933,0,0,0
7,2,0.0,0.0,7.5e-05,0.0,2018-08-01 11:22:26,true,2018-07-28 05:06:15,"4 days, 6 hours, 16 minutes, and 11 seconds",1.0,7.5e-05,13333,0.003161689198881704,616852581614855,0,0,0
0,0,0.0,0.0,0.0,0.0,2018-08-01 12:23:28,false,2018-08-01 11:23:28,1 hour,0.0,0.0,0,0.00028742629080743,0,0,0,0


In [ ]:
#   eth 
#   keys 
#   mask 
#   ico  

player_round = (1..rID_).map do |r|
    player_round = (1..pID_).map do |p|
        $fomo_contract.call.plyr_rnds_(p,r)
    end
    puts "== round #{r} =="
    puts player_round.map {|x| x.map {|y| y/1e18}.join(" - ")}.join("\n")
end

nil

== round 1 ==
0.001 - 13.333162041616681 - 0.0 - 0.0
0.0 - 0.0 - 0.0 - 0.0
0.0 - 0.0 - 0.0 - 0.0
0.0 - 0.0 - 0.0 - 0.0
0.0 - 0.0 - 0.0 - 0.0
0.0 - 0.0 - 0.0 - 0.0
0.0 - 0.0 - 0.0 - 0.0
0.0 - 0.0 - 0.0 - 0.0
0.0 - 0.0 - 0.0 - 0.0
== round 2 ==
0.0 - 0.0 - 0.0 - 0.0
0.0 - 0.0 - 0.0 - 0.0
0.07606488906250031 - 1013.0000000000041 - 0.058944753002591645 - 0.0
0.011343308750000202 - 151.0000000000027 - 0.008786434060603615 - 0.0
0.003756847656250101 - 50.00000000000134 - 0.002307077697649762 - 0.0
0.0 - 0.0 - 0.0 - 0.0
0.0 - 0.0 - 0.0 - 0.0
0.0 - 0.0 - 0.0 - 0.0
0.0 - 0.0 - 0.0 - 0.0
== round 3 ==
0.0 - 0.0 - 0.0 - 0.0
0.0 - 0.0 - 0.0 - 0.0
0.0075757890625001 - 101.00000000000132 - 0.004358051210273654 - 0.0
0.0 - 0.0 - 0.0 - 0.0
0.0 - 0.0 - 0.0 - 0.0
0.0 - 0.0 - 0.0 - 0.0
0.0 - 0.0 - 0.0 - 0.0
0.0 - 0.0 - 0.0 - 0.0
0.0 - 0.0 - 0.0 - 0.0
== round 4 ==
0.0 - 0.0 - 0.0 - 0.0
0.0 - 0.0 - 0.0 - 0.0
0.0451031718750001 - 601.0000000000014 - 0.031235907923519237 - 0.0
0.11301526640496762 - 1502.638

In [ ]:
## fees_ and potSplit_

# fees_[0] = F3Ddatasets.TeamFee(30,6);   //50% to pot, 10% to aff, 2% to com, 1% to pot swap, 1% to air drop pot
# fees_[1] = F3Ddatasets.TeamFee(43,0);   //43% to pot, 10% to aff, 2% to com, 1% to pot swap, 1% to air drop pot
# fees_[2] = F3Ddatasets.TeamFee(56,10);  //20% to pot, 10% to aff, 2% to com, 1% to pot swap, 1% to air drop pot
# fees_[3] = F3Ddatasets.TeamFee(43,8);   //35% to pot, 10% to aff, 2% to com, 1% to pot swap, 1% to air drop pot

# potSplit_[0] = F3Ddatasets.PotSplit(15,10);  //48% to winner, 25% to next round, 2% to com
# potSplit_[1] = F3Ddatasets.PotSplit(25,0);   //48% to winner, 25% to next round, 2% to com
# potSplit_[2] = F3Ddatasets.PotSplit(20,20);  //48% to winner, 10% to next round, 2% to com
# potSplit_[3] = F3Ddatasets.PotSplit(30,10);  //48% to winner, 10% to next round, 2% to com

## calc key and eth
# iWantXKeys
# calcKeysReceived


In [ ]:
amount = 1000000000/1e18


1.0e-09

In [ ]:
# =================. Withdraw =============

In [ ]:
tx = $contract.transact_and_wait.withdraw

#<Ethereum::Transaction:0x000055a9b849b328 @mined=true, @connection=#<Ethereum::HttpClient:0x000055a9b8c66328 @id=0, @log=false, @batch=nil, @formatter=#<Ethereum::Formatter:0x000055a9b8c66300>, @gas_price=22000000000, @gas_limit=4000000, @host="a768-123-117-180-47.ngrok.io", @port=80, @proxy=nil, @ssl=false, @uri=#<URI::HTTP http://a768-123-117-180-47.ngrok.io>, @default_account="0xf39fd6e51aad88f6f4ce6ab8827279cfffb92266">, @id="0x9f2833c60bde8a589c4981ac81d65433978ca9619d51ea53df179c4f75d51f54", @input="0x3ccfd60b0000000000000000000000000000000000000000000000000000000000000000", @input_parameters=[]>

In [ ]:
$client.eth_get_balance("0xaCC961474eb17C103BBed96FC8f0E4eEac91c475")["result"].to_i(16)

99998553855455157046

In [ ]:
$client.eth_get_balance(contract_addr)["result"].to_i(16)

55331223153569919

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=ebde59c2-d67b-4e31-ad3b-1bdd07014691' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>